# Simple power spectrum estimation from an input dataset
This example shows how to estimate the power spectrum from a set of data files using an Optimal Quadratic Estimator (OQE) approach.

In [1]:
%matplotlib inline
from pyuvdata import UVData
import hera_pspec as hp
import numpy as np
import matplotlib.pyplot as plt
import copy
import os
import itertools
from funcsigs import signature
from hera_pspec.data import DATA_PATH

## Loading the input data: forming power spectra from adjacent time integrations

The input data are specified as a list of `UVData` objects, which are then packaged into a `PSpecData` class. This class is responsible for collecting the data and covariances together and performing the OQE power spectrum estimation.

At least two `UVData` objects must be specified, these could be different datasets, or just copies of a single dataset, given the use-case. In what follows, we will use only one data set and produce two copies of it, but will shift the second dataset by one time integration and interleave it relative to the first, such that we can form auto-baseline power spectra without noise-bias. 

In [2]:
# select the data file to load
dfile = os.path.join(DATA_PATH, 'zen.all.xx.LST.1.06964.uvA')

# Load into UVData objects
uvd = UVData()
uvd.read_miriad(dfile)

# Check which baselines are available
print(uvd.get_antpairs())

[(24, 24), (24, 25), (24, 37), (24, 38), (24, 39), (25, 25), (25, 37), (25, 38), (25, 39), (37, 37), (37, 38), (37, 39), (38, 38), (38, 39), (39, 39)]


## Define a cosmology

Here we will instantiate a `hera_pspec.conversions.Cosmo_Conversions` object that will define the cosmology we adopt throughout this notebook. **All calculations that depend on a cosmology will be tied to the one we adopt here: the various objects in `hera_pspec` will pass this specific object between themselves, which starts by attaching the cosmology to a `PSpecBeamUV` object.** Further on in the notebook, we will see how we can explicitly overwrite the adopted cosmology and appropriately re-calculate the necessary variables.

In [3]:
# Instantiate a Cosmo Conversions object
# we will need this cosmology to put the power spectra into cosmological units
cosmo = hp.conversions.Cosmo_Conversions()
print(cosmo)


Cosmo_Conversions object at <0x122f5e250>
Om_L : 0.6844; Om_b : 0.0491; Om_c : 0.2644; Om_M : 0.3135; Om_k : 0.0021; H0 : 67.2700


Instantiate a beam object, and attach the cosmo conversions object onto it.

In [ ]:
# List of beamfile to load. This is a healpix map.
beamfile = os.path.join(DATA_PATH, 'HERA_NF_dipole_power.beamfits')

# intantiate beam and pass cosmology, if not fed, a default Planck cosmology will be assumed
uvb = hp.pspecbeam.PSpecBeamUV(beamfile, cosmo=cosmo)
from pyuvdata import UVBeam, utils as uvutils
pol_array =  uvb.primary_beam.polarization_array
pol= uvutils.polstr2num('yy')
stokes_p_ind = np.where(np.isin(pol_array, pol))[0][0]
print stokes_p_ind
print np.shape(uvb.primary_beam.data_array)
freq = np.linspace(120*1e6,150*1e6,500)
beam_res = uvb.primary_beam._interp_freq(freq)

In [27]:
print np.shape(beam_res[0])

(1, 1, 4, 500, 12288)


Next convert from Jy units to mK. This involves calculating the effective beam area (see HERA Memo #27 and #43), which can be done with the beam object we instantiated earlier.

In [5]:
# find conversion factor from Jy to mK
Jy_to_mK = uvb.Jy_to_mK(np.unique(uvd.freq_array), pol='XX')

# reshape to appropriately match a UVData.data_array object and multiply in!
uvd.data_array *= Jy_to_mK[None, None, :, None]

Configure data and instantiate a `PSpecData` object, while also feeding in the beam object.

In [6]:
# slide the time axis of uvd by one integration
uvd1 = uvd.select(times=np.unique(uvd.time_array)[:-1:2], inplace=False)
uvd2 = uvd.select(times=np.unique(uvd.time_array)[1::2], inplace=False)

# Create a new PSpecData object, and don't forget to feed the beam object
beamfile_Q = os.path.join(DATA_PATH, 'Gaussian_beam_freq_0.beamfits')
bm_Q  = hp.pspecbeam.PSpecBeamUV(beamfile_Q)
bls   = [(24, 25)] 
spw_ranges=[(600, 700)]
ds_Q  = hp.pspecdata.PSpecData(dsets=[uvd, uvd], wgts=[None, None],beam=bm_Q, labels=['red', 'blue'])
uvp = ds_Q.pspec(bls, bls, (0, 1), [('xx', 'xx')], spw_ranges=spw_ranges, input_data_weight='identity',
                                       norm='I', taper='none', verbose=True, exact_norm=False)


#print ds_Q.spw_range
#print ds_Q.delays()[0]

#Q_sample = ds_Q.get_Q(0) #Get Q matrix for 0th delay mode                                                                                             
#np.testing.assert_equal(np.shape(Q_sample), (spw_ranges[0][1] - spw_ranges[0][0], spw_ranges[0][1] - spw_ranges[0][0])) #Check for the right shape


Setting spectral range: (600, 700)

Using polarization pair: ('xx', 'xx')

(bl1, bl2) pair: ((24, 25), (24, 25))
pol: (-5, -5)
  Building G...
  Building q_hat...
  Computing and multiplying scalar...
  Normalizing power spectrum...


In [7]:
uvd = UVData()
uvd.read_miriad(os.path.join(DATA_PATH,"zen.2458042.17772.xx.HH.uvXA"))
beamfile_Q = os.path.join(DATA_PATH, 'Gaussian_beam_freq_0.beamfits')
bl1_Q  = (25,39)
bl2_Q  = (25,39)
bm_Q   = hp.PSpecBeamUV(beamfile_Q)

ds_t    = hp.PSpecData(dsets=[uvd, uvd], wgts=[None, None], beam=bm_Q)

uvp_new = ds_t.pspec([bl1_Q], [bl2_Q], (0, 1), [('xx', 'xx')], input_data_weight='identity',
                               norm='I', taper='none', verbose=True, exact_norm=True)
uvp_ext = ds_t.pspec([bl1_Q], [bl2_Q], (0, 1), [('xx', 'xx')], input_data_weight='identity',
                               norm='I', taper='none', verbose=True, exact_norm=False)

spw         = 0
blp         = (bl1_Q, bl2_Q)
key         = (spw, blp, 'xx')
power_real_new  = (np.real(uvp_new.get_data(key)))
power_real_ext  = (np.real(uvp_ext.get_data(key)))

print np.shape(power_real_new)
print np.shape(power_real_ext)


Setting spectral range: (0, 64)

Using polarization pair: ('xx', 'xx')

(bl1, bl2) pair: ((25, 39), (25, 39))
pol: (-5, -5)
 which may lead to normalization instabilities.
  Building G...
  Building q_hat...


TypeError: tuple indices must be integers, not tuple

In [ ]:
diff = (power_real_new[50]-power_real_ext[50])/power_real_ext[50]
print diff

## Phase second `dset` to first `dset`

Because we are forming power spectra between datasets that are offset in LST there will be some level of decoherence (and therefore signal loss) of the EoR signal. For short baselines and small LST offsets this is typically negligible, but it is still good to try to recover what coherency we can, simply by phasing (i.e. fringe-stopping) the datasets before forming the power spectra. This can be done with the `rephase_to_dset` method, and can only be done once. 

In [ ]:
# Because the LST integrations are offset by more than ~15 seconds we will get a warning
# but this is okay b/c it is still **significantly** less than the beam-crossing time and we are using short
# baselines...

# here we phase all datasets in dsets to the zeroth dataset
ds.rephase_to_dset(0)

In [ ]:
# change units of UVData objects
ds.dsets[0].vis_units = 'mK'
ds.dsets[1].vis_units = 'mK'

## Estimating the power spectrum for a handful of baseline pairs (auto-baseline pspec)
Estimate the power spectrum for a handful baseline pairs between the two datasets in `ds.dsets`. You can specify which baselines are included in the power spectrum estimate, which datasets to use, what freq channels to use, and how the estimate should be weighted.

In [ ]:
# Specify which baselines to include
baselines = [(24,25), (37,38), (38,39)]

### Read the docstring! Here are the first few lines...

In [ ]:
print("ds.pspec{}".format(signature(ds.pspec)))
print('\n'.join(ds.pspec.__doc__.split('\n')[:30]))

Call the `PSpecData.pspec` function to use the OQE framework. The result is a `UVPSpec` object that holds all of the power spectra and their meta-data.

In [ ]:
# we will use the baselines list to produce 3 power spectra
# whose data will be drawn from the dsets[0] and dsets[1]
# across two spectral windows with identity weighting and a blackman-harris taper
uvp = ds.pspec(baselines, baselines, (0, 1), [('xx', 'xx')], spw_ranges=[(300, 400), (600,721)], input_data_weight='identity',
               norm='I', taper='blackman-harris', verbose=True)

## Making sense of the meta-data

The fundamental unit in the `UVPSpec` object is a single delay spectrum, indexed by a spectral window selection (`spw`), a baseline-pair selection (`blpair`) and a polarization (`pol`). Spectral windows are marked by their index (see `spw_array`), polarization are marked by their pol string (`xx`) or pol integer (`-5`), and a baseline-pair is marked by its blpair integer, which is simply the antenna numbers put into length-3 integers and concatenated. For example, the baseline-pair `((100, 200), (300, 400))` would have a blpair integer `100200300400`.

To access particular slices of the data in a `UVPSpec` object, the user should interface with the `get_data` method, which takes a selection of `spw`, `blpair` and `pol` as arguments. See the example below.

In [ ]:
# let's get the delay spectra from spw 0, blpair ((24, 25), (24, 25)) and xx pol
key = (0, ((24, 25), (24, 25)), 'xx')

# output should be shape (Ntimes, Ndlys)
print(uvp.get_data(key).shape)

# we can also access data by feeding a dictionary
key = {'pol':'xx', 'spw': 0, 'blpair':((24, 25), (24, 25))}
print(uvp.get_data(key).shape)

There is also metadata telling you things like the data weighting, normalization, tapering, units, telescope location, LST and JD time stamps, etc.

In [ ]:
# get power spectrum units
print("pspec units: ", uvp.units)

# get weighting
print("pspec weighting: ", uvp.weighting)

Also, importantly, the cosmology we adopted originally and passed through to the beam and `PSpecData` objects was passed through to this `UVPSpec` object.

In [ ]:
print(uvp.cosmo)

## Plotting

Here is an example of how to do some simple and quick plotting of power spectra, which includes getting the power spectra and the delay bins of their bandpowers.

In [ ]:
# plot power spectrum of spectral window 1
fig, ax = plt.subplots(figsize=(12,8))

spw = 1
blp =((24, 25), (24,25))
key = (spw, blp, 'xx')
dlys = uvp.get_dlys(spw) * 1e9
power = np.abs(np.real(uvp.get_data(key)))

p1 = ax.plot(dlys, power.T)
ax.set_yscale('log')
ax.grid()
ax.set_xlabel("delay [ns]", fontsize=14)
ax.set_ylabel(r"$P(k)\ \rm [mK^2\ h^{-3}\ Mpc^3]$", fontsize=14)
ax.set_title("spw : {}, blpair : {}, pol : {}".format(*key), fontsize=14)

## Form redundant baseline spectra (auto & cross baseline pspec)

Above we looked at auto-baseline power spectra. Here we will show how to form power spectra between groups of redundant baselines. To do this, we use the `pspecdata.construct_blpairs` helper function to construct the `baselines1` and `baselines2` lists that we then feed into `PSpecData.pspec`. This takes keywords `exclude_auto_bls`, which will remove all baselines paired with itself from the final `blpairs` list, as well as `exclude_permutations`, which will use a combination instead of a permutation to form the baseline-pairs from the input `baselines` list.

In [ ]:
# baselines is a redundant baseline group
baselines = [(24,25), (37,38), (38,39)]

# calculate all baseline pairs from group
baselines1, baselines2, blpairs = hp.utils.construct_blpairs(baselines, exclude_auto_bls=True, 
                                                                 exclude_permutations=True)

# Inspect baseline pairs
print(blpairs)

Now we feed these lists through to `pspec`

In [ ]:
uvp = ds.pspec(baselines1, baselines2, (0, 1), [('xx', 'xx')], spw_ranges=[(300, 400), (600,721)], input_data_weight='identity', 
               norm='I', taper='blackman-harris', verbose=True)

In [ ]:
# plot power spectrum of a cross-baseline pspectra from spectral window 1
fig, ax = plt.subplots(figsize=(12,8))

spw = 1
blp =((24, 25), (37, 38))
key = (spw, blp, 'xx')
dlys = uvp.get_dlys(spw) * 1e9
power = np.abs(np.real(uvp.get_data(key)))

_ = ax.plot(dlys, power.T)
ax.set_yscale('log')
ax.grid()
ax.set_xlabel("delay [ns]", fontsize=14)
ax.set_ylabel(r"$P(k)\ \rm [mK^2\ h^{-3}\ Mpc^3]$", fontsize=14)
ax.set_title("spw : {}, blpair : {}, pol : {}".format(*key), fontsize=14)

## Convert to $\Delta^2(k)$

You can convert the power spectrum data from $P(k)$ format to "unitless" power spectrum $\Delta^2(k)$ by scaling by the cosmological wave-vectors cubed. This can be done by hand with the `get_kparas()` and `get_kperps()` methods, or can be done automatically using the `convert_to_deltasq()` method.

The cosmology used for this calculation will be the one stored in the `UVPSpec` object, which, recall, is the same cosmology we adopted originally.

In [ ]:
# make a copy of the UVPSpec object
dsq = copy.deepcopy(uvp)

# how to get the wave vectors for spw = 1 by hand
k_perp, k_para = dsq.get_kperps(1, little_h=True), dsq.get_kparas(1, little_h=True)

# convert to deltasq automatically
dsq.convert_to_deltasq()

# see that units have automatically changed
print("dsq units: ",dsq.units)

In [ ]:
# plot deltasq power spectrum of spw 1
fig, ax = plt.subplots(figsize=(12,8))

spw = 1
blp =((24, 25), (37, 38))
key = (spw, blp, 'xx')
power = np.abs(np.real(dsq.get_data(key)))

_ = ax.plot(k_para, power.T)
ax.set_yscale('log')
ax.grid()
ax.set_xlabel(r"$k_{\parallel}\ h\ Mpc^{-1}$", fontsize=14)
ax.set_ylabel(r"$\Delta^2(k)\ \rm [mK^2]$", fontsize=14)
ax.set_title("spw : {}, blpair : {}, pol : {}".format(*key), fontsize=14)

## Plotting Thermal Noise Curves

Using the `hera_pspec.noise` module, you can plot analytic thermal noise power spectra given a $T_{\rm sys}$ and a `UVPSpec` object, which carries with it the integration time and number of incoherent averages for **each delay spectrum** it contains.

To do this, all you need to do as the user is to interface with the `UVPSpec.generate_noise_spectra()` method. Given a selection of spectral window, polarization, system temperature and optionally which blpairs to compute for, it will return a dictionary of thermal noise curves (in either $P(k)$ or $\Delta^2(k)$ depending on `form` kwarg) for each baseline-pair in the `UVPSpec` object.

In [ ]:
# generate noise curves for spw 0, pol 'xx' with a Tsys of 300 K
spw = 1
pol = 'xx'
Tsys = 300
P_N = uvp.generate_noise_spectra(spw, pol, Tsys)

Have generated the `Sense` object, we can now call `generate_noise_spectra` to produce noise curves. The result is a dictionary with baseline-pair integers as keys and ndarrays as values.

In [ ]:
# plot power spectrum of spw 1 with noise curves
fig, ax = plt.subplots(figsize=(12,8))

blp = ((24, 25), (37, 38))
key = (spw, blp, pol)
power = np.abs(np.real(uvp.get_data(key)))

_ = ax.plot(k_para, power.T)
ax.set_yscale('log')
ax.grid()
ax.set_xlabel(r"$k_\parallel\ \rm h\ Mpc^{-1}$", fontsize=14)
ax.set_ylabel(r"$P(k)\ \rm [mK^2\ h^{-3}\ Mpc^3]$", fontsize=14)
ax.set_title("spw : {}, blpair : {}, pol : {}".format(*key), fontsize=14)

# plot noise curves for this baseline-pair
ax.set_prop_cycle(None)
_ = ax.plot(k_para, P_N[uvp.antnums_to_blpair(blp)].T, ls='--', lw=3)

## Incoherent Averaging

The `UVPSpec` object supports incoherent averaging across baseline-pairs and/or time using the `UVPSpec.average_spectra()` method. If the baseline-pairs are grouped by redundancy, this is equivalent to binning onto a single 3D $k_x, k_y, k_\parallel$ space. If the baseline-pairs are grouped by baseline separation, this is equivalent to cylindrical binning in $k_\perp$ and $k_\parallel$. Spherical binning to form a true 1D power spectrum is currently not supported.

Below we will incoherently average across all baseline-pairs in the `UVPSpec` object, knowing that we only constructed 14.6 m power spectra, so we are reasonably justified in doing so.

In [ ]:
# form the baseline-pair group, which will be a single group 
# consisting of all baseline-pairs in the object
blp_group = [sorted(np.unique(uvp.blpair_array))]

# average spectra with inplace = False and assign to a new "uvp2" object
uvp2 = uvp.average_spectra(blpair_groups=blp_group, time_avg=True, inplace=False)

We can then plot the averaged power spectrum, along with a new noise curve which will know to account for the averaging based on the changes to the `integration_array` and `nsample_array`. The colored lines show the same un-averaged pspectra from before, while the black line shows the averaged power spectrum, along with its new thermal noise estimate.

In [ ]:
# plot power spectrum of spw 1
fig, ax = plt.subplots(figsize=(12,8))

spw = 1
blp = ((24, 25), (37, 38))
pol = 'xx'
key = (spw, blp, pol)
k_para = uvp.get_kparas(spw)
power = np.abs(np.real(uvp.get_data(key)))
P_N = uvp.generate_noise_spectra(spw, pol, 300)
P_N = P_N[uvp.antnums_to_blpair(blp)]

avg_power = np.abs(np.real(uvp2.get_data(key)))
avg_P_N = uvp2.generate_noise_spectra(spw, pol, 300)
avg_P_N = avg_P_N[uvp2.antnums_to_blpair(blp)]

_ = ax.plot(k_para, power.T)
ax.plot(k_para, avg_power.T, color='k')
ax.plot(k_para, avg_P_N.T, color='k', ls='--', lw=3)
ax.set_yscale('log')
ax.grid()
ax.set_xlabel(r"$k_\parallel\ \rm h\ Mpc^{-1}$", fontsize=14)
ax.set_ylabel(r"$P(k)\ \rm [mK^2\ h^{-3}\ Mpc^3]$", fontsize=14)
ax.set_title("spw : {}, blpair : {}, pol : {}".format(*key), fontsize=14)

ax.set_prop_cycle(None)
_ = ax.plot(k_para, P_N.T, ls='--', lw=3)

## Fold the power spectra

The power spectra are natively across positive and negative delays, but often we make plots with only positive delay bins. In addition, averaging bandpowers with identical negative with positive bandpowers gives us an extra factor of sqrt 2 sensitivity, which we can do with the `fold_spectra()` method.

In [ ]:
uvp.fold_spectra()

In [ ]:
uvp2.fold_spectra()

In [ ]:
# plot power spectrum of spw 1
fig, ax = plt.subplots(figsize=(12,8))

spw = 1
blp =((24, 25), (37, 38))
pol = 'xx'
key = (spw, blp, pol)
k_para = uvp.get_kparas(spw)
power = np.abs(np.real(uvp.get_data(key)))
P_N = uvp.generate_noise_spectra(spw, pol, 300)
P_N = P_N[uvp.antnums_to_blpair(blp)]

avg_power = np.abs(np.real(uvp2.get_data(key)))
avg_P_N = uvp2.generate_noise_spectra(spw, pol, 300)
avg_P_N = avg_P_N[uvp2.antnums_to_blpair(blp)]

_ = ax.plot(k_para, power.T)
ax.plot(k_para, avg_power.T, color='k')
ax.plot(k_para, avg_P_N.T, color='k', ls='--', lw=3)
ax.set_yscale('log')
ax.grid()
ax.set_xlabel(r"$k_\parallel\ \rm h\ Mpc^{-1}$", fontsize=14)
ax.set_ylabel(r"$P(k)\ \rm [mK^2\ h^{-3}\ Mpc^3]$", fontsize=14)
ax.set_title("spw : {}, blpair : {}, pol : {}".format(*key), fontsize=14)

ax.set_prop_cycle(None)
_ = ax.plot(k_para, P_N.T, ls='--', lw=3)

## Re-defining the cosmology

We started by adopting a single cosmology, which was automatically passed between the various `hera_pspec` objects without user intervention in order to preserve self-consistency. If, however, we would like to adopt a *new cosmology*, we need to re-compute certain cosmology-dependent quantities, namely the normalization of the power spectrum scalar (see HERA Memos #27 and #43). We can do this with the `UVPSpec.set_cosmology` object.

In [ ]:
# define a new cosmology with no dark energy
new_cosmo = hp.conversions.Cosmo_Conversions(Om_L = 0.0)

# attempt to overwrite the current UVPSpec cosmology:
# this will fail b/c one already exists and overwrite=False
uvp.set_cosmology(new_cosmo)

In [ ]:
# now set overwrite = True
uvp.set_cosmology(new_cosmo, overwrite=True)

## Perform data selection on `UVPSpec`

We can select out parts of the data using the `select` function.

In [ ]:
# select all baseline-pairs containing the baseline (24, 25)
# only_pairs_in_bls means to select only blpairs that have baseline1 _and_ baseline2 in the bls list
uvp2 = uvp.select(bls=[(24, 25)], inplace=False, only_pairs_in_bls=False)

# check than only baseline-pairs with a (24, 25) in them are kept
print("baseline pairs in uvp2: \n", map(lambda blp: uvp2.blpair_to_antnums(blp), np.unique(uvp2.blpair_array)))

## Write to HDF5

Write to disk using the `write_hdf5` method. Easy as that.

In [ ]:
uvp.write_hdf5("pspec.hdf5", overwrite=True)

## Read from HDF5

Read the whole file, just the meta-data, or a data selection for partial I/O.

Frist, let's **read the whole file** and look at some meta-data.

In [ ]:
# read the whole file
uvp2 = hp.uvpspec.UVPSpec()
uvp2.read_hdf5("pspec.hdf5")

In [ ]:
# print some meta-data
print("pols : ", uvp2.pol_array)
print("number of baseline-pairs: ",uvp2.Nblpairs)

In [ ]:
# Ensure data_array exists b/c we read the whole file
print("data_array in uvp2 : ", hasattr(uvp2, 'data_array'))

Now read **only the meta-data**.

In [ ]:
# read only the meta-data
uvp2.read_hdf5("pspec.hdf5", just_meta=True)

In [ ]:
# print some meta-data
print("pols : ", uvp2.pol_array)
print("number of baseline-pairs: ",uvp2.Nblpairs)

In [ ]:
# Ensure data_array doesn't exist b/c we only loaded the meta-data
print("data_array in uvp2 : ", hasattr(uvp2, 'data_array'))

Now do **partial loading of certain baseline pairs**.

In [ ]:
# partial load on certain baseline-pairs 
uvp2.read_hdf5("pspec.hdf5", just_meta=False, bls=[(37, 38), (38, 39)], only_pairs_in_bls=True)

# print baseline pairs
print("baseline pairs in uvp2: \n", map(lambda blp: uvp2.blpair_to_antnums(blp), np.unique(uvp2.blpair_array)))

Now do **partial loading of certain times**. To do this, we need to feed the exact float of the times we want, but how do we get these to float precision in the first place without loading the whole file? This is where the `just_meta` load comes in handy.

First load just the metadata, then do a partial load using the times you desire from the metadata.

In [ ]:
# read only meta
uvp2.read_hdf5("pspec.hdf5", just_meta=True)

# now only load in one of the time integrations
uvp2.read_hdf5("pspec.hdf5", times=np.unique(uvp2.time_avg_array)[:1])

# check that only one time integration exists, but that all baseline pairs exist
print("number of unique times: ", len(np.unique(uvp2.time_avg_array)))
print("number of baseline-pairs: ",uvp2.Nblpairs)

In [ ]:
if os.path.exists("pspec.hdf5"): os.remove("pspec.hdf5")

## Forming Pseudo-Stokes Power Spectra

Forming polarized pseudo-Stokes power spectra is identical to forming linear-dipole polarization power spectra, except that we need pseudo-Stokes visibilities and beam models. See the `Forming_PseudoStokes_Vis` notebook on details on how to do that. Here we will show how to form pseudo-I spectra. 

In [ ]:
# form pseudoI visibility
dset1 = os.path.join(DATA_PATH, 'zen.all.xx.LST.1.06964.uvA')  # xx miriad file
dset2 = os.path.join(DATA_PATH, 'zen.all.yy.LST.1.06964.uvA')  # yy miriad file
uvdI = hp.pstokes.construct_pstokes(dset1=dset1, dset2=dset2, pstokes='pI')

In [ ]:
# List of beamfile to load. This is a healpix map for both XX and YY, and can therefore make an I map
beamfile = os.path.join(DATA_PATH, 'HERA_NF_pstokes_power.beamfits')

# intantiate beam and pass cosmology, if not fed, a default Planck cosmology will be assumed
uvb = hp.pspecbeam.PSpecBeamUV(beamfile, cosmo=cosmo)

In [ ]:
# find conversion factor from Jy to mK for XX pol
Jy_to_mK = uvb.Jy_to_mK(np.unique(uvdI.freq_array), pol='pI')

# reshape to appropriately match a UVData.data_array object and multiply in!
uvdI.data_array *= Jy_to_mK[None, None, :, None]

# changeunits of UVData objects
uvdI.vis_units = 'mK'

In [ ]:
# instantiate a PSpecData object
ds = hp.PSpecData(dsets=[uvdI, uvdI], wgts=[None, None], beam=uvb)

In [ ]:
# baselines is a redundant baseline group
baselines = [(24,25), (37,38), (38,39)]

# calculate all baseline pairs from group
baselines1, baselines2, blpairs = hp.utils.construct_blpairs(baselines, exclude_auto_bls=True, 
                                                                 exclude_permutations=True)

# Inspect baseline pairs
print(blpairs)

In [ ]:
uvp = ds.pspec(baselines1, baselines2, (0, 1), [('pI', 'pI')], spw_ranges=[(300, 400)], 
               input_data_weight='identity', norm='I', taper='blackman-harris', verbose=True)

In [ ]:
# plot power spectrum of spectral window 0
fig, ax = plt.subplots(figsize=(12,8))

spw = 0
blp = ((24, 25), (37, 38))
key = (spw, blp, 'pI')
dlys = uvp.get_dlys(spw) * 1e9
power = np.abs(np.real(uvp.get_data(key)))

p1 = ax.plot(dlys, power.T)
ax.set_yscale('log')
ax.grid()
ax.set_xlabel("delay [ns]", fontsize=14)
ax.set_ylabel(r"$P(k)\ \rm [mK^2\ h^{-3}\ Mpc^3]$", fontsize=14)
ax.set_title("spw : {}, blpair : {}, pol : {}".format(*key), fontsize=14)

## Using Custom Polarized Beams

In the case that you have information on polarized beams that can't be put into a `UVBeam` object, you can use the custom beam capability of `hera_pspec.pspecbeam.PSpecBeamFromArray`, which will allow you to create a `PSpecBeam` object from custom $\Omega_P$ and $\Omega_{PP}$ arrays. See the `PSpecBeam` tutorial to see how to do this. 

One can then use this object in the same way as above to create polarized power spectra with custom polarized beams.